In [227]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [228]:
maping = lambda x: torch.exp(-x)

In [229]:
X = torch.rand((100, 2), dtype=float)
Y = torch.sum(maping(X), dim=1)

In [230]:
X.shape, Y.shape

(torch.Size([100, 2]), torch.Size([100]))

In [231]:
class DenseLayer:
    def __init__(self, n_neurons, lr=5e-3):
        self.n_neurons = n_neurons
        self.has_initialized = False
        self.lr = lr
        
    def forwardPass(self, input_):
        if self.has_initialized is False:
            self.weights = torch.rand(self.n_neurons, input_.shape[1], dtype=float).requires_grad_()
            self.bias = torch.rand(torch.Size([]), dtype=float)
            self.input = input_
            self.has_initialized = True
        self.output = (self.weights @ self.input.T + self.bias).T
        
    def calcLoss(self, loss):
        self.loss = loss.calcLoss(self.output)
        
    def backPropogation(self):
        self.loss.backward(retain_graph=True)
        
    def update(self):
        self.weights.data -= self.weights.grad.data * self.lr
        self.weights.grad.zero_()

In [238]:
class MeanSquaredLoss:
    def __init__(self, y):
        self.y = y
    
    def calcLoss(self, pred_val):
        loss = F.mse_loss(pred_val, self.y).sqrt()
        print(f'Current Loss: {loss}')
        return loss

In [239]:
class Sequential:
    def __init__(self, layers, lossFn=MeanSquaredLoss):
        self.layers = layers
        self.layers_rev = layers[::-1]
        self.lossFn = lossFn
        
    def fit(self, x, y, no_of_epochs):
        history = dict({
            'loss':[]
        })
        for _ in range(no_of_epochs):
            for idx, layer in enumerate(self.layers):
                if idx == 0:
                    layer.forwardPass(x)
                else:
                    layer.forwardPass(self.layers[idx-1].output)
                    
            for idx, layer in enumerate(self.layers_rev):
                if idx == 0:
                    layer.calcLoss(self.lossFn(y))
                    history['loss'].append(layer.loss.item())
                    layer.backPropogation()
                
                layer.update()
            
        return history

In [ ]:
Y = Y.reshape(Y.shape[0], 1)

In [ ]:
model = Sequential([
    DenseLayer(4),
    DenseLayer(1),], lossFn = MeanSquaredLoss)

In [ ]:
history = model.fit(X, Y, 400)

In [ ]:
plt.plot(range(len(history['loss'])), history['loss'], label="Training")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc='best')